In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
#os.environ['ARROW_PRE_0_15_IPC_FORMAT'] = '1'
from pyspark.pandas import read_csv
import pyspark.pandas as ps
# means is for items, df is for collab

In [2]:
ps.set_option('compute.default_index_type', 'distributed')
big_data = False
path ='data/100k_a.csv' if  not big_data else 'data/full_a.csv'
df = read_csv(path, names=['userId','streamId','streamerName','timeStart','timeStop'])

In [3]:
df.head()

,userId,streamId,streamerName,timeStart,timeStop
0,1,33842865744,mithrain,154,156
1,1,33846768288,alptv,166,169
2,1,33886469056,mithrain,587,588
3,1,33887624992,wtcn,589,591
4,1,33890145056,jrokezftw,591,594


In [4]:
df['interactionTime']=(df['timeStop'] - df['timeStart'])*10 # units are 10 min

In [5]:
df.head()

,userId,streamId,streamerName,timeStart,timeStop,interactionTime
0,1,33842865744,mithrain,154,156,20
1,1,33846768288,alptv,166,169,30
2,1,33886469056,mithrain,587,588,10
3,1,33887624992,wtcn,589,591,20
4,1,33890145056,jrokezftw,591,594,30


In [6]:
df= df.drop(columns=['timeStart','timeStop'])

In [7]:
tmax=df['interactionTime'].max()

In [8]:
tmin=df['interactionTime'].min()

In [9]:
df['interactionTime']= (df['interactionTime']- tmin)/(tmax-tmin)

In [10]:
df.head()

,userId,streamId,streamerName,interactionTime
0,1,33842865744,mithrain,0.010417
1,1,33846768288,alptv,0.020833
2,1,33886469056,mithrain,0.000000
3,1,33887624992,wtcn,0.010417
4,1,33890145056,jrokezftw,0.020833


In [11]:
# goal is to scale to 1-100 for implict rating/confidence
# MAYBE INVESTIGATE DIFFERENT SCALING IN FUTURE
df['interactionTime'] = (df['interactionTime'] * 99) +1

In [12]:
df.head()

,userId,streamId,streamerName,interactionTime
0,1,33842865744,mithrain,2.03125
1,1,33846768288,alptv,3.06250
2,1,33886469056,mithrain,1.00000
3,1,33887624992,wtcn,2.03125
4,1,33890145056,jrokezftw,3.06250


In [13]:
import sys
df['streamerId']=df['streamerName'].apply(lambda x: hash(x) & sys.maxsize) # prevent negative numbers

In [14]:
df.head()

,userId,streamId,streamerName,interactionTime,streamerId
0,1,33842865744,mithrain,2.03125,6237787267506800044
1,1,33846768288,alptv,3.06250,8455602530264088426
2,1,33886469056,mithrain,1.00000,6237787267506800044
3,1,33887624992,wtcn,2.03125,5324476659562100413
4,1,33890145056,jrokezftw,3.06250,6731343583494858722


In [15]:
len(df['streamerId'].unique()) == len(df['streamerName'].unique())

True

In [16]:
df = df.drop(columns=['streamerName'])

In [17]:
df.head()

,userId,streamId,interactionTime,streamerId
0,1,33842865744,2.03125,6237787267506800044
1,1,33846768288,3.06250,8455602530264088426
2,1,33886469056,1.00000,6237787267506800044
3,1,33887624992,2.03125,5324476659562100413
4,1,33890145056,3.06250,6731343583494858722


In [18]:
means=df.groupby(['streamerId'],as_index=False)['interactionTime'].mean()
means=means.rename(columns={'streamerId':'streamerId','interactionTime':'avgInteractionTime'})

In [19]:
means.head()

,streamerId,avgInteractionTime
0,5652140647009628783,2.324660
1,392638289472592052,2.250000
2,6900454007790282002,2.938563
3,2406168726911974914,6.981250
4,2271347389753034915,2.630040


In [20]:
numStreams = df.groupby(['streamerId'],as_index=False).size()
numStreams.max()

45144

In [21]:
means=means.join(numStreams,on='streamerId')

In [22]:
means= means.rename(columns={'streamerId':'streamerId','avgInteractionTime':'avgInteractionTime', 0:'interactionCounts'})
means.head()

,streamerId,avgInteractionTime,interactionCounts
0,5652140647009628783,2.324660,239
1,392638289472592052,2.250000,33
2,6900454007790282002,2.938563,1323
3,2406168726911974914,6.981250,10
4,2271347389753034915,2.630040,93


### better for smaller in memory data
### df.to_pandas().to_csv('data/test.csv')

In [23]:
# used to ensure no oom error with pandas
df.to_spark().coalesce(1).write.format('csv').option('header', 'true').save('data/collab')

In [24]:
means.to_spark().coalesce(1).write.format('csv').option('header', 'true').save('data/item')